
# 一些测试程序用于研究关键代码

In [1]:
teachers=["谢博","谢博士","杨佳庆","杨家庆"]

teacherAlais={"谢博":"谢凯年","谢博士":"谢凯年","修培宁":"谢凯年","杨家庆":"杨佳庆"}
mxwarning=""

def getRealName(nickname):
    global teacherAlais
    global mxwarning
    try:
        tals=teacherAlais[nickname]
        if tals != "谢凯年":
            mxwarning+="老师名字写错系统自动更正,"
            print(mxwarning)
    except:
        tals=nickname
    return tals

for teachername in teachers:
    print(getRealName(teachername))

谢凯年
谢凯年
杨佳庆
老师名字写错系统自动更正,
杨佳庆


In [2]:
import re

timestr="a2018-10-06少"
gsearchstart=re.search("(\d+)-(\d+)-(\d+)",timestr)
print(gsearchstart)
gstart=gsearchstart.groups()
print(gstart)

<_sre.SRE_Match object; span=(1, 11), match='2018-10-06'>
('2018', '10', '06')


# 清洗有课互联原始数据

In [3]:
import csv
import re
import datetime

f=open("有课互联签到原始数据.csv",encoding='utf-8', mode = 'r')
lines=f.readlines()
csvrows=csv.reader(lines,delimiter=',')
startpattern='^(^|上午|下午|:)(\d+)(:|点|：)(\d+|到|~|-)'
endpattern='(到|~|-)(\d+)(:|点|：)(\d+|$)(\ +|$)$'
dianpingpattern='(^|.+)(点评:|点评：|特点:|特点：|点评：|老师评语：|老师评语:|特点：|总结经验:|评价:|评价：)(.+|$)'
datepattern="(\d+)-(\d+)-(\d+)"

teacherAlais={"谢博":"谢凯年","谢博士":"谢凯年","修培宁":"谢凯年","杨家庆":"杨佳庆"}
mxwarning=""

def getRealName(nickname):
    global teacherAlais
    global mxwarning
    try:
        tals=teacherAlais[nickname]
        if tals != "谢凯年":
            mxwarning+="老师名字写错系统自动更正,"
            print(mxwarning)
    except:
        tals=nickname
    return tals

#从签到备注中删除补签正常签等域,如果过短，根据有课互联前10列数据猜测补充
def getQiandaobeizhu(ykvols,qiandao_org):    
    global mxwarning
    qcols=qiandao_org.split("|")
    qdbz=""
    for qcol in qcols:
        #删除 补签 正常签 以及首个空字符
        if qcol == "补签" or qcol == "正常签" or qcol=="" or qcol=="\n": 
            continue
        else:
            qdbz+=qcol
            qdbz+="|"
 #   print(qdbz)
    qcols=qdbz.split("|")
    #如果缺乏上课地址以及上课时间信息，从有课互联记录里恢复
     #只有姓名和签到时间
 #   print(qcols)
 #   print(len(qcols))
    if len(qcols)==3:
        mxwarning+="记录不全自动补齐需要复查，"
        #添加上课地址为有课互联校区地址
        qdbz+=ykvols[9]
        qdbz+="|"
        #添加上课时间为零点开始的有课互联销课时间
        qdbz+="00:00"
        qdbz+="-"
        qdbz+=str(int(float(ykvols[5])))+":00"
        qdbz+="|"
        qdbz+="学习内容：1，2，3 特点：|"
    #只有姓名和签到时间和签到地点
    if len(qcols)==4:
        mxwarning+="记录不全自动补齐需要复查，"
        #添加上课时间为零点开始的有课互联销课时间
        qdbz+="00:00"
        qdbz+="-"
        qdbz+=str(int(float(ykvols[5])))+":00"
        qdbz+="|"
        qdbz+="学习内容：1，2，3 特点：|"
  #  print(qdbz)
    return qdbz

#从签到备注中获得教师和助教信息
def getTeacherAndTA(qiandaobeizhu):
    global mxwarning
    #    print(qiandaobeizhu)
    qcols=qiandaobeizhu.split("|")
#    print(qcols)
    if len(qcols) <2:
        return 'NA','NA'    
    tryteacher=qcols[0]
    TA="NA"
    #老师名字长度大于4，说明有助教，填上助教信息,主课老师位置排第一,第二个为助教，第三个不计入系统
    if(len(tryteacher)<4):
        teacher=tryteacher
    else:
        teas=tryteacher.split("、")
        teacher=teas[0]  
        if len(teas)>1:
            TA=teas[1]
        if len(teas)>2:
            mxwarning+="超过一个助教不计入系统，"
    teacher=getRealName(teacher)
    TA=getRealName(TA)
    return teacher,TA

#从签到备注中获得上课日期信息
def getTeachDate(qiandaobeizhu):
    global mxwarning
    counter=0;
#    print(qiandaobeizhu)
    qcols=qiandaobeizhu.split("|")
#    print(qcols)
    year="1980"
    month="01"
    day="01"
    tdate=year+"-"+month+"-"+day
  # 查找签到备注中每一个字段看有没有年月日时间信息，有的话计算为上课日期
    for qcol in qcols:        
        gsearchstart=re.search(datepattern,qcol)
        if gsearchstart is None:
            counter+=1  
        else:
            gstart=gsearchstart.groups()
            year=gstart[0]
            month=gstart[1]
            day=gstart[2]
            if int(year) <2015:
   #            print(" less than 2015")
                mxwarning+="日期小于2015年系统自动纠正为2018请核实，"
                year="2018"
            elif year=="20180":
                year="2018"
                mxwarning+="日期中的年20180自动纠正为2018下次请注意,"
            tdate=year+"-"+month+"-"+day
            #验证日期合法性
            try:
                objDate = datetime.datetime.strptime(tdate,"%Y-%m-%d")
   #            转换为标准日期格式输出
                tdate=str(objDate)[0:10]
            except:                
                tdate="2018-01-01"
                mxwarning+="日期不合法系统自动纠正为2018-01-01请核实，"
            
             
                print(tdate)
            return tdate,counter
    mxwarning+="没找到日期严重错误系统猜测为第2列，"
    #没找到则猜测日期数据为第二列
    return tdate,1

#获得上课地点，去除补签正常签等信息后，上课地点永远在上课日期之后
def getTeachAddr(qiandaobeizhu):
    global mxwarning
    teachaddr="NA"
    qcols=qiandaobeizhu.split("|")
#    print(qcols)
    if len(qcols) <2:
        return 'NA'  
    #先找上课日期，得到上课日期的位置
    teachdate,pos=getTeachDate(qiandaobeizhu)
    if pos >5:
        return 'NA'
 #   print(pos)
    teachaddr=qcols[pos+1]
    return teachaddr

#获得上课时间
def getTeachTime(qiandaobeizhu):
    global mxwarning
    #先找上课日期，得到上课日期的位置
    teachdate,pos=getTeachDate(qiandaobeizhu)
    qcols=qiandaobeizhu.split("|")
    #正常情况下日期后面的字段是地点，再后面是上课时间段，因此pos+2
 #   print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
 #   print(qcols)
 #   print(pos)
    
    #看能否找到时间模式，不能则选紧接着的字段作为上课时间段字段
    trytimestr=qcols[pos+2]
    gsearchstart=re.search(startpattern,trytimestr)
    if gsearchstart is None:
        tpos=pos+1
        mxwarning+="地点和日期顺序错误，"
    else:
        tpos=pos+2
 #   print(tpos)
    ttimestr=qcols[tpos]
    #   ttimestr="16:00~18:00,19:00~20:00"
    timestrs=ttimestr.split(",")
    teachtime=""
    for timestr in timestrs:
        #timestr 16:00~18:00
  #      print(timestr)
        teachtimesegstart="23:00"
        teachtimesegend="24:00"
        gsearchstart=re.search(startpattern,timestr)
        gstart=gsearchstart.groups()
        gsearchend=re.search(endpattern,timestr)
        if gsearchend is None:
            gsearchend=gsearchstart
        gend=gsearchend.groups()
        
        if not gstart[3].isdigit():
            startm='0'
        else:
            startm=gstart[3]
  
    
        if not gend[3].isdigit():
            endm='0'
        else:
            endm=gend[3]

   #     print(gstart)
   #     print(gend)
        
        sgstart=gstart[1]+":"+startm
        sgend=gend[1]+":"+endm

    #    print(sgstart)
    #    print(sgend)
        objDate = datetime.datetime.strptime(sgstart,"%H:%M")
        teachtimesegstart=objDate.strftime("%H:%M")
        objDate = datetime.datetime.strptime(sgend,"%H:%M")
        teachtimesegend=objDate.strftime("%H:%M")
       
        teachtime+=teachtimesegstart+"-"+teachtimesegend+"*"
    return teachtime

#获得上课内容
def getTeachCont(qiandaobeizhu):
    global mxwarning
    qdbz=qiandaobeizhu.split("|")
    qdbztc=qdbz[4]
#    print(qdbztc)
    qdzs=qdbztc.split("\n")
    teachc=""
    for qdz in qdzs:        
        gs=re.search(dianpingpattern,qdz)
#        print("!!!!!!!!!!!!!!!!!!!qdz!!!!!!!!!!!!!!!!!")
#        print(qdz)
        if gs is None:
            teachc+=qdz
        else:
            gsg=gs.groups()
#            print("!!!!!!!!!!!!!!for gsg!!!!!!!!!!!!!!!!!!")
#            print(gsg)
            teachc+=gsg[0]
    return teachc
  
#获得学生评价
def getStuComment(qiandaobeizhu):
    global mxwarning
    qdbz=qiandaobeizhu.split("|")
    qdbztc=qdbz[4]
#    print(qdbztc)
    qdzs=qdbztc.split("\n")
    teachc=""
    stucomment=""
    for qdz in qdzs:        
        gs=re.search(dianpingpattern,qdz)
#        print("!!!!!!!!!!!!!!!!!!!qdz!!!!!!!!!!!!!!!!!")
#        print(qdz)
        if gs is None:
            teachc+=qdz
        else:
            gsg=gs.groups()
#            print("!!!!!!!!!!!!!!for gsg!!!!!!!!!!!!!!!!!!")
#            print(gsg)
            stucomment+=gsg[2]
            continue
        if stucomment != "":
            stucomment+=qdz 
    return stucomment
    
#获得其它信息
def getOtherComments(qiandaobeizhu):
    global mxwarning
    othercomments=mxwarning
    return othercomments

#主程序从这里开始
rowcounter=0
lenrow=0
rows=[]

#将签到备注转换为表格中的11列~17列
for row in csvrows:
    newrow=row[0:11]
    if(rowcounter==0):
        newrow[10:18]=["授课教师","助教","授课日期","授课地点","授课时间","其它备注","授课内容","学生点评"]
    else:
        newrow[10:18]=["UNKNOWN_TEACHER","UNKNOWN_TA","UNKOWN_DATE","UNKONW_ADDR","UNKOWN_TIME","OTHERS","UNKONW_CONT","UNKONW_STUCOMENTS"]
        #原始签到备注
        mxwarning=""
        qiandao_org=row[11]
        #删除 补签 正常签等标记
        qiandaobeizhu=getQiandaobeizhu(row[0:10],qiandao_org)
#        print(qiandaobeizhu)
        #得到教师和助教信息
        (teacher,TA)=getTeacherAndTA(qiandaobeizhu)
        newrow[10:12]=[teacher,TA]
        #得到授课日期
        datestring,counter=getTeachDate(qiandaobeizhu)
        newrow[12]=datestring
        #得到授课地点
        teachaddr=getTeachAddr(qiandaobeizhu)
        newrow[13]=teachaddr
        #得到授课时间
        timestring=getTeachTime(qiandaobeizhu)
        newrow[14]=timestring
        #获得其它信息        
        othercomment=getOtherComments(qiandaobeizhu)
        newrow[15]=othercomment
        #得到授课内容
        teachcont=getTeachCont(qiandaobeizhu)
        newrow[16]=teachcont
        #获得学生评价
        stucomment=getStuComment(qiandaobeizhu)
        newrow[17]=stucomment
 #   print(newrow)
 #   print('\n')
 #   print('\n')
    rows.append(newrow)
    lenrow+=len(row)
    rowcounter=rowcounter+1
    print(rowcounter)
    
print("总共",rowcounter,"行，总共",lenrow,"列,每行",lenrow/rowcounter,"列")
print(rows[0:10])
print(rows[-10:-1])


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
老师名字写错系统自动更正,
219
老师名字写错系统自动更正,
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270


# 结果存入有课互联签到加工数据

In [4]:

fw=open("M01_ykhl_washed.csv",'w',encoding='utf8',newline='')
csvwriter=csv.writer(fw)
csvwriter.writerows(rows)
fw.close()
